# Classification and Regresssion Trees(Classification)

## 一、概念

决策树是一种经典且当前应用依然十分活跃的算法，用简单的一句话概括决策树的训练过程就是不停划分变量空间最后得到函数映射。就像分段函数一样:

\[
sign(x)=\begin{cases}
1 & x>0\\
-1 & x\le 0
\end{cases}
\]

我们可以看到这样的分类具有完备和互斥地性质，这意味着对于样本空间内的输入都有且仅有一个输出。

如果你熟悉数据结构，那么决策树是通常一个完全二叉树，如果不熟悉也没关系。

决策树有以下要点：

1. 组织形式——树，

2. 划分准则——信息增益、基尼系数等，本质都是衡量“不纯度”

3. 增长方式——通过不断地选择变量进行划分然后生成子树

4. 剪枝——防止过拟合

譬如我们有一组数据，判断病人是否可能患病，决策树可能就会通过判断年龄是否大于50，血压血糖是否大于某个值来进行判断，也因此决策树具有很强的可解释性，展示起来十分直观

## 二、训练

### 1.组织形式

树依靠节点之间的关系来组织。因为暂时不方便传图，所以不赘述。根节点为1，父节点和子节点之间有如下关系式：

\[
parent(i) = \lfloor \frac i2 \rfloor \\
left-child(i) = 2i \\
right-child(i) = 2i+1
\]

### 2.划分准则

衡量不纯度的指标有很多，不同的决策树算法会选用不同的指标，比如经典的ID3树和C4.5选用信息熵作为划分准则，而CART分类树选用基尼系数。

基尼系数：假设有K个类，样本点属于第i类的概率为$p_i$，概率分布的基尼系数定义为：

\[
Gini(p) = \sum_{i=1}^K p_i(1-p_i) = 1 - \sum_{i=1}^K p_i^2
\]

对于给定的样本集合D，其基尼系数为：

\[
Gini(D) = 1 - \sum_{i=1}^K \left(\frac {|C_i|}{|D|} \right)^2
\]

其中$C_i$是样本中属于第i类的样本子集，K是类的个数。

如果样本集合D根据特征A是否去某一可能值a被分割成$D_1$和$D_2$两个子集，则在特征A取值a的情况下集合D的基尼系数定义为：

\[
Gini(D,A=a) = \frac{|D_1|}{|D|}Gini(D_1) + \frac{|D_2|}{|D|}Gini(D_2)
\]



### 3.增长方式

有了组织形式和划分准则之后，我们就要考虑如何让这棵树“生长”。

以二分类为例，易知基尼系数的最大值是0.5，当且仅当取值0或1的概率都为0.5，也就是变量取值0和1的概率一样大。此时模型的输出可以视为随机的，应用在样本集合中也就是0和1的样本一样多，此时进行预测的结果和抛硬币无异，这显然不是我们想要的。

而当基尼系数的取到最小值0时，可知有取值为0或1的概率中一个为0，一个为1，也就是变量取值是确定的，此时模型的输出是固定的，应用在样本集合中也就是只有0或1的样本，此时进行预测的结果是确定的，这才是我们希望得到的。

因此我们可以发现基尼系数越大表示样本D的不确定性越大，所以我们希望**随着树的生长，基尼系数以最快的速度下降**。


以下表的数据为例，变量A是连续变量，变量B是离散变量：

|索引|变量A|变量B|标签|
|----|-----|-----|----|
|1   |2.1  |1    |1   |
|2   |2.2  |1    |1   |
|3   |1.8  |1    |0   |
|4   |2.0  |1    |0   |
|5   |1.6  |1    |1   |
|6   |2.8  |2    |1   |
|7   |1.3  |2    |0   |
|8   |0.1  |2    |0   |
|9   |3.8  |2    |1   |
|10  |2.9  |2    |1   |
|11  |0.5  |3    |0   |
|12  |2.7  |3    |1   |
|13  |0.9  |3    |0   |
|14  |1.4  |3    |0   |
|15  |1.2  |3    |0   |

#### 对离散变量

\[
Gini(D,B=1) = \frac{5}{15}\times\left(1-\left(\frac35\right)^2-\left(\frac25\right)^2\right) + \frac{10}{15}\times \left(1-\left(\frac{4}{10}\right)^2-\left(\frac{6}{10}\right)^2\right)=\frac{12}{25}\\
Gini(D,B=2) = \frac{12}{25}\\
Gini(D,B=3) = \frac{32}{75} < \frac{36}{75} = \frac{12}{25}
\]

于是我们将B是否等于3作为划分条件生成两个子节点，或者直接将B作为叶节点，B等于3则预测0，B不等于3则预测1.

#### 对连续变量

连续变量的情况就会稍复杂一些，这由连续变量的特点决定，连续变量是稠密的，很可能一个样本集合内的某一连续变量没有两个相同的值，此时划分按照变量是否等于某个值进行则极为不合理，于是划分取值则按照变量是否小于（小于等于）某个值进行。这和大于等于（大于）是完全等价的。

决策树的特点决定，决策树从根节点到叶节点的划分变量和划分取值都**以使基尼系数下降最大为目标**，这意味着决策树是确定的，每一个划分都是最优的。而对于连续变量的最优划分，则需要遍历连续变量的每个取值（大多数情况下也是遍历整个样本），然后选出最优划分条件。

以表中数据为例，连续变量A的最优划分条件是$I\{A\le 2.0\}$（我是这么设计的...）

\[
Gini(D,A\le 2.0) = \frac{9}{15}\left(1-\left(\frac19\right)^2 - \left(\frac89 \right)^2\right) + \frac{6}{15}\left(1-\left(\frac66\right)^2 - \left(\frac06 \right)^2\right) = \frac{16}{135}
\]

#### 停止

每次划分节点时，使用一个变量，在此节点的子树下，当前划分变量不会再次出现，当划分变量用完时树的生长就会自然停止。

这样生成的树极有可能会过拟合，有两种解决方法：一是早停法，即是不把所有变量用来生成划分条件，只选择部分变量进行划分，然而这样生成的决策树不具有确定性，因为随机种子不同，选择的变量也不同；另一种解决方法则是对决策树进行剪枝。

### 4.剪枝

剪枝相关还不熟练，暂时跳过。

## 三、应用

直接上真实数据集可能比较玄学，先用先前的数据集进行验证计算基尼系数的函数等：

In [54]:
import numpy as np
import pandas as pd

test_data = pd.DataFrame({
    'A':[2.1, 2.2, 1.8, 2.0, 1.6, 2.8, 1.3, 0.1, 3.8, 2.9, 0.5, 2.7, 0.9, 1.4, 1.2],
    'B':[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3],
    'label':[1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]
})
test_data

,A,B,label
0,2.1,1,1
1,2.2,1,1
2,1.8,1,0
3,2.0,1,0
4,1.6,1,1
5,2.8,2,1
6,1.3,2,0
7,0.1,2,0
8,3.8,2,1
9,2.9,2,1


In [55]:
def Gini(data, labels, index, target='label'):
    data = data.loc[index]
    n = data.shape[0]

    gini = 1
    
    for label in labels:
        p = np.sum(data[target] == label)/n
        gini -= p*p
    
    return gini

print(Gini(data=test_data, labels=[0, 1], index=np.linspace(0, 14, 15)))

0.4977777777777777


In [62]:
def conditional_gini(data, labels, index, split_variable, split_value, target='label', discrete=True):
    data = data.loc[index]
    n = data.shape[0]
    if discrete:
        index1 = data[data[split_variable]==split_value].index
        index2 = data[data[split_variable]!=split_value].index
        
        gini1 = Gini(data=data, labels=labels, index=index1, target=target)
        gini2 = Gini(data=data, labels=labels, index=index2, target=target)
        
        gini = len(index1)/n*gini1 + len(index2)/n*gini2
        
        return gini
    else:
        index1 = data[data[split_variable]<=split_value].index
        index2 = data[data[split_variable]>split_value].index
        
        gini1 = Gini(data=data, labels=labels, index=index1, target=target)
        gini2 = Gini(data=data, labels=labels, index=index2, target=target)
        
        gini = len(index1)/n*gini1 + len(index2)/n*gini2

        return gini

print(conditional_gini(data=test_data, labels=[0,1], index=np.linspace(0, 14, 15), split_variable='B', split_value=3))
print(32/75)

0.4266666666666665
0.4266666666666667


这次选的数据集是汽车数据集：http://archive.ics.uci.edu/ml/datasets/Car+Evaluation 

一共有六个变量：

- buying：购买的价格，有四个取值，vhigh、high、med、low

- maint：保养的价格，有四个取值，vhigh、high、med、low

- doors：门的数量，有四个取值，2、3、4、5more

- persons：载荷人数，有三个取值，2、4、more

- lug_boot：后备箱的尺寸，有三个取值，small、med、big

- safety：安全性，有三个取值，low、med、high

还有一个标签：

- acceptability：可接受程度，有四个取值，unacc、acc、good、vgood

就直觉而言，门的数量和载荷人数按照连续变量进行处理更为合适，也有助于演示连续变量的划分。

In [63]:
car = pd.read_csv('car.data', header=None, names=['buying','maint','doors','persons','lug_boot','safety','acc'])
car.head()

,buying,maint,doors,persons,lug_boot,safety,acc
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


计算训练集和测试集的基尼系数

In [59]:
car = car.replace(['more', '5more'], 6)
n = car.shape[0]

np.random.seed(2099)
index = np.random.permutation(n)
train_index = index[0: int(0.7*n)]
test_index = index[int(0.7*n): n]

labels = np.unique(car['acc'])
print(gini(data=car, labels=labels, index=train_index))
print(gini(data=car, labels=labels, index=test_index))

0.46404653272499263
0.4407096795749942


计算以buying为划分变量，‘vhigh’为划分值的基尼系数

In [64]:
print(conditional_gini(data=car, labels=labels, index=train_index, split_variable='buying', split_value='vhigh', target='acc'))

0.45587846574195173


In [ ]:
def split_value(data, index, variable, discrete=True):
    data = data.iloc[index]
    
    if discrete:
        values=np.unique(data[variable])
        for value in values:
            

In [17]:
def build_tree(data, index, split_variable, to_split):
    if len(to_split) > 1:
        

69

In [36]:
car[car['buying']=='vhigh']

,buying,maint,doors,persons,lug_boot,safety,acc
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


In [34]:
len(split)

0